# 쿠팡 데이터 크롤링

___

## Global functions, parameters

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
import time

driver = webdriver.Chrome()

In [2]:
def getURL(seed, params=None, select='a', num_retries=2):
    html = requests.get(seed, params=params, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getURL(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    links = dom.select(select)
    
    return ['http://www.coupang.com' + link['href'] for link in links if link.has_attr('href')]

In [3]:
headers = {
    "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    "accept-encoding":"false"
}

___

### 과자
1. link
2. img
3. brand
4. title: 제품명(있는 그대로)
5. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
6. price: 가격(int)
7. rating: 평점
8. replyCount: 상품평 갯수
9. reply: 20개

In [4]:
params_snack = {
    "listSize": "120",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "1",
    "channel": "user",
    "fromComponent": "Y",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221916",
    "rating": "0",
}

In [5]:
seed_snack = 'http://www.coupang.com/np/categories/222016'

In [6]:
links_snack = []
links_snack.extend(getURL(seed_snack, params_snack, select='ul.baby-product-list a.baby-product-link'))
links_snack = links_snack[:100]

In [236]:
snackList = []

In [237]:
def getInfo_snack(url, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_snack(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', '')) > 0:
        link = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        brand = dom.select('a.prod-brand-name')[0].get_text()
        title = dom.select('h2.prod-buy-header__title')[0].get_text()
        # product = 
        price = int(dom.select('span.total-price')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        replyCount = int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', ''))
        
        driver.get(url)
        body = driver.find_element_by_tag_name('body')
        while True:
            try:
                driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
                break
            except NoSuchElementException:
                body.send_keys(Keys.PAGE_DOWN)
                driver.implicitly_wait(0.5)
                
        rating = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
        rating = rating.get_attribute('data-rating')
        replyResult = []
        buttonList = []
        buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
        
        def getReply(pagenum):
            try:
                element = WebDriverWait(driver, 5).until(lambda x: x.find_elements_by_css_selector('.js_reviewArticleListContainer article div.sdp-review__article__list__review__content.js_reviewArticleContent')) 
                replyResult.extend(reply.text.replace('\n', ' ') for reply in element)
                time.sleep(1)
            except TimeoutException:
                return None
            
            while True:
                try:
                    buttonList = []
                    buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
                    break
                except WebDriverException:
                    body.send_keys(Keys.PAGE_DOWN)
                    driver.implicitly_wait(0.5)
            if len(buttonList) > 0:
                buttonList[pagenum].click()
            else:
                return None
        
        if len(buttonList) > 0:
            for i in range(1, 5):
                getReply(i)
                time.sleep(0.5)
    
        result = {
            'link': link, 
            'img': img, 
            'brand': brand, 
            'title': title, 
            'price': price, 
            'rating': rating, 
            'optionTitle': None,
            'optionPrice': None,
            'replyCount': replyCount, 
            'replyResult': replyResult
        }
        snackList.append(result)
        return 1
    
    else:
        return 0

In [238]:
for i in range(100):
    if getInfo_snack(links_snack[i]) == 0:
        print('Skip: ' + links_snack[i])
    time.sleep(0.3)

Skip: http://www.coupang.com/vp/products/13705765?itemId=57173218&vendorItemId=3089582979
Skip: http://www.coupang.com/vp/products/36807682?itemId=135730618&vendorItemId=3289389004
Skip: http://www.coupang.com/vp/products/18274522?itemId=73563189&vendorItemId=3120782803
Skip: http://www.coupang.com/vp/products/17623945?itemId=70623827&vendorItemId=3114223534
Skip: http://www.coupang.com/vp/products/2400458?itemId=3998892&vendorItemId=3004797294
Skip: http://www.coupang.com/vp/products/96477934?itemId=296996220&vendorItemId=3730702348
Skip: http://www.coupang.com/vp/products/42593455?itemId=154271433&vendorItemId=3359501206


In [283]:
len(snackList)

93

평점 없는것, 품절인것 제외.

___

### 분유
1. link
2. img
3. brand
4. origin: 원산지(국내 / 해외)
5. title: 제품명(있는 그대로)
6. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
7. stage: 분유 단계(int)
8. type: 분말 / 액상
9. price: 가격(int)
10. rating: 평점
11. replyCount: 상품평 갯수
12. reply: 20개

In [45]:
params_milkPowder = {
    "listSize": "120",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "1",
    "channel": "user",
    "fromComponent": "N",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "221895",
    "rating": "0",
}

In [46]:
seed_milkPowder = 'http://www.coupang.com/np/categories/221995'

In [259]:
links_milkPowder = []
links_milkPowder.extend(getURL(seed_milkPowder, params_milkPowder, select='ul.baby-product-list a.baby-product-link'))
links_milkPowder = links_milkPowder[:100]

In [266]:
milkPowderList = []

In [261]:
def getInfo_milkPowder(url, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_milkPowder(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', '')) > 0:
        link = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        brand = dom.select('a.prod-brand-name')[0].get_text()
        title = dom.select('h2.prod-buy-header__title')[0].get_text()
        # product = 
        index = 0
        stage = 0
        for index in range(len(dom.select('li.prod-attr-item'))):
            try:
                stage = int(dom.select('li.prod-attr-item')[index].get_text().replace('\n', '').replace('단계: ', '').replace('단계', ''))
            except ValueError:
                index += 1
            
        if stage == 0:
            stageIndex = title.find('단계')
            try:
                stage = int(title[stageIndex-1])
            except ValueError:
                return 0
        
        if '액상분유' in title:
            milkPowderType = '액상분유'
        elif '스틱분유' in title:
            milkPowderType = '스틱분유'
        else:
            milkPowderType = '원통형분유'
        price = int(dom.select('span.total-price')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        replyCount = int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', ''))
        
        driver.get(url)
        body = driver.find_element_by_tag_name('body')
        while True:
            try:
                driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
                break
            except NoSuchElementException:
                body.send_keys(Keys.PAGE_DOWN)
                driver.implicitly_wait(0.5)
        
        origin = driver.find_elements_by_css_selector('.prod-delivery-return-policy-table.essential-info-table tbody tr td')[1].text
        rating = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
        rating = rating.get_attribute('data-rating')
        replyResult = []
        buttonList = []
        buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
        
        def getReply(pagenum):
            try:
                element = WebDriverWait(driver, 5).until(lambda x: x.find_elements_by_css_selector('.js_reviewArticleListContainer article div.sdp-review__article__list__review__content.js_reviewArticleContent')) 
                replyResult.extend(reply.text.replace('\n', ' ') for reply in element)
                time.sleep(1)
            except TimeoutException:
                return None
            buttonList = []
            while True:
                try:
                    buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
                    break
                except WebDriverException:
                    body.send_keys(Keys.PAGE_DOWN)
                    driver.implicitly_wait(0.5)
            if len(buttonList) > 0:
                buttonList[pagenum].click()
            else:
                return None
        
        if len(buttonList) > 0:
            for i in range(1, 5):
                getReply(i)
                time.sleep(0.5)
    
        result = {
            'link': link, 
            'img': img, 
            'brand': brand, 
            'origin': origin,
            'title': title, 
            'stage': stage,
            'type': milkPowderType,
            'price': price, 
            'rating': rating, 
            'optionTitle': None,
            'optionPrice': None,
            'replyCount': replyCount, 
            'replyResult': replyResult
        }
        milkPowderList.append(result)
        return 1
    
    else:
        return 0
    
    return None

In [269]:
for i in range(78, 100, 1):
    if getInfo_milkPowder(links_milkPowder[i]) == 0:
        print('Skip: ' + links_milkPowder[i])
    time.sleep(0.3)

Skip: http://www.coupang.com/vp/products/108994041?itemId=329738403&vendorItemId=3808867816
Skip: http://www.coupang.com/vp/products/854046?itemId=1963928&vendorItemId=3001924247


In [284]:
len(milkPowderList)

98

2개 스킵: 단계 이상해서, 단계 옵션으로 주어져서 스킵됨.

___

### 장난감
1. link
2. img
3. category: 감각발달완구, 아기체육관, 러닝홈/플레이매트, 아기걸음마, 모빌/딸랑이/치발기, 오뚝이
4. title: 제품명(있는 그대로)
5. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
6. price: 가격(int)
7. rating: 평점
8. replyCount: 상품평 갯수
9. reply: 20개

In [4]:
categoryList_toy = ['감각발달완구', '아기체육관/러닝홈', '아기걸음마', '모빌', '딸랑이', '치발기', '오뚝이']

In [5]:
params_toy = {
    "listSize": "120",
    "brand": "",
    "offerCondition": "",
    "filterType": "",
    "isPriceRange": "false",
    "minPrice": "",
    "maxPrice": "",
    "page": "1",
    "channel": "user",
    "fromComponent": "Y",
    "selectedPlpKeepFilter": "",
    "sorter": "bestAsc",
    "filter": "",
    "component": "",
    "rating": "0",
}

In [6]:
seed_toy_list = ['http://www.coupang.com/np/categories/349712',
                 'http://www.coupang.com/np/categories/349704',
                 'http://www.coupang.com/np/categories/349710',
                 'http://www.coupang.com/np/categories/349687',
                 'http://www.coupang.com/np/categories/349689',
                 'http://www.coupang.com/np/categories/349690',
                 'http://www.coupang.com/np/categories/349694']

In [7]:
toyComponentList = [349612, 349604, 349610, 349587, 349589, 349590, 349594]

In [8]:
links_toy = {
    categoryList_toy[0]: None,
    categoryList_toy[1]: None,
    categoryList_toy[2]: None,
    categoryList_toy[3]: None,
    categoryList_toy[4]: None,
    categoryList_toy[5]: None,
    categoryList_toy[6]: None
}

for i in range(7):
    params_toy['component'] = toyComponentList[i]
    temp = []
    temp.extend(getURL(seed_toy_list[i], params_toy, select='ul.baby-product-list a.baby-product-link'))
    temp = temp[:100]
    links_toy[categoryList_toy[i]] = temp

In [9]:
for (k, v) in links_toy.items():
    print(k, len(v))

감각발달완구 100
아기체육관/러닝홈 100
아기걸음마 100
모빌 100
딸랑이 100
치발기 100
오뚝이 100


In [12]:
links_toy['감각발달완구'][0]

'http://www.coupang.com/vp/products/5174059?itemId=23450011&vendorItemId=3034349958'

In [13]:
links_toy['아기체육관/러닝홈'][0]

'http://www.coupang.com/vp/products/18534606?itemId=74488762&vendorItemId=3153259364'

In [10]:
toyList = []

In [274]:
def getInfo_toy(url, category, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_toy(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', '')) > 0:
        link = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        category = category
        title = dom.select('h2.prod-buy-header__title')[0].get_text()
        # product = 
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        replyCount = int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', ''))
        
        driver.get(url)
        body = driver.find_element_by_tag_name('body')
        while True:
            try:
                driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
                break
            except NoSuchElementException:
                body.send_keys(Keys.PAGE_DOWN)
                driver.implicitly_wait(0.5)
        
        rating = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
        rating = rating.get_attribute('data-rating')
        replyResult = []
        buttonList = []
        buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
        
        def getReply(pagenum):
            try:
                element = WebDriverWait(driver, 5).until(lambda x: x.find_elements_by_css_selector('.js_reviewArticleListContainer article div.sdp-review__article__list__review__content.js_reviewArticleContent')) 
                replyResult.extend(reply.text.replace('\n', ' ') for reply in element)
                time.sleep(1)
            except TimeoutException:
                return None
            buttonList = []
            while True:
                try:
                    buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
                    break
                except WebDriverException:
                    body.send_keys(Keys.PAGE_DOWN)
                    driver.implicitly_wait(0.5)
            if len(buttonList) > 0:
                buttonList[pagenum].click()
            else:
                return None
        
        if len(buttonList) > 0:
            end = len(buttonList)
            if end > 5:
                end = 5
            for i in range(1, end):
                getReply(i)
                time.sleep(0.5)
    
        result = {
            'link': link, 
            'img': img, 
            'category': category,
            'title': title, 
            'price': price, 
            'rating': rating, 
            'optionTitle': None,
            'optionPrice': None,
            'replyCount': replyCount, 
            'replyResult': replyResult
        }
        toyList.append(result)
        return 1
    
    else:
        return 0
    
    return None

In [286]:
for link in links_toy[categoryList_toy[0]]:
    getInfo_toy(link, categoryList_toy[0])

In [288]:
for link in links_toy[categoryList_toy[1]]:
    getInfo_toy(link, categoryList_toy[1])

In [291]:
for link in links_toy[categoryList_toy[2]]:
    getInfo_toy(link, categoryList_toy[2])

In [293]:
for link in links_toy[categoryList_toy[3]]:
    getInfo_toy(link, categoryList_toy[3])

In [295]:
for link in links_toy[categoryList_toy[4]]:
    getInfo_toy(link, categoryList_toy[4])

In [297]:
for link in links_toy[categoryList_toy[5]]:
    getInfo_toy(link, categoryList_toy[5])

In [298]:
for link in links_toy[categoryList_toy[6]]:
    getInfo_toy(link, categoryList_toy[6])

In [299]:
len(toyList)

441

___

## 기저귀
1. link
2. img
3. brand
4. origin: 원산지(국내 / 해외)
5. title: 제품명(있는 그대로)
6. product: 보여줄 제품명(브랜드 + 연령대 + 형태)
7. stage: {“신생아”:1,  “소형”:2, “중형”:3, “대형”:4, “특대형”:5, “점보”:6}
8. type: 밴드 / 팬티
9. price: 가격(int)
10. rating: 평점
11. replyCount: 상품평 갯수
12. reply: 20개

In [36]:
size_diaper = {
    '1': 'http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244488%40DEFAULT&component=221863&rating=0',
    '2': 'http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244489%40DEFAULT&component=221863&rating=0',
    '3': 'http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244490%40DEFAULT&component=221863&rating=0',
    '4': 'http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244491%40DEFAULT&component=221863&rating=0',
    '5': 'http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244492%40DEFAULT&component=221863&rating=0',
    '6': 'http://www.coupang.com/np/categories/221963?listSize=60&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=1%23attr_11880%244493%40DEFAULT&component=221863&rating=0'
}

In [38]:
links_diaper = {
    '1': None,
    '2': None,
    '3': None,
    '4': None,
    '5': None,
    '6': None
}

for i in range(1, 7, 1):
    temp = []
    temp.extend(getURL(size_diaper[str(i)], select='ul.baby-product-list a.baby-product-link'))
    temp = temp[:100]
    links_diaper[i] = temp
    print(links_diaper[i])

['http://www.coupang.com/vp/products/85604303?itemId=270702790&vendorItemId=3655707093', 'http://www.coupang.com/vp/products/99298423?itemId=303913110&vendorItemId=3749461407', 'http://www.coupang.com/vp/products/59817328?itemId=205901638&vendorItemId=3488670486', 'http://www.coupang.com/vp/products/33247691?itemId=124880500&vendorItemId=3254565792', 'http://www.coupang.com/vp/products/24421767?itemId=95089650&vendorItemId=3169935702', 'http://www.coupang.com/vp/products/70040853?itemId=234305214&vendorItemId=3572950022', 'http://www.coupang.com/vp/products/34829455?itemId=129631216&vendorItemId=3270753056', 'http://www.coupang.com/vp/products/22253971?itemId=86595282&vendorItemId=3201932407', 'http://www.coupang.com/vp/products/24026944?itemId=93556955&vendorItemId=3164946783', 'http://www.coupang.com/vp/products/65722204?itemId=221777229&vendorItemId=3537317780', 'http://www.coupang.com/vp/products/1399012?itemId=2926856&vendorItemId=3015429797', 'http://www.coupang.com/vp/products/1

['http://www.coupang.com/vp/products/85604243?itemId=270702627&vendorItemId=3655706775', 'http://www.coupang.com/vp/products/85604295?itemId=270702771&vendorItemId=3655707061', 'http://www.coupang.com/vp/products/95879139?itemId=287362791&vendorItemId=3705465021', 'http://www.coupang.com/vp/products/59817330?itemId=205901640&vendorItemId=3488670488', 'http://www.coupang.com/vp/products/99298424?itemId=303913112&vendorItemId=3749461405', 'http://www.coupang.com/vp/products/85604241?itemId=270702626&vendorItemId=3655706774', 'http://www.coupang.com/vp/products/99298442?itemId=303913131&vendorItemId=3749461506', 'http://www.coupang.com/vp/products/99298403?itemId=303913068&vendorItemId=3749461322', 'http://www.coupang.com/vp/products/6154565?itemId=27518782&vendorItemId=3040811564', 'http://www.coupang.com/vp/products/12601732?itemId=53452555&vendorItemId=3083555577', 'http://www.coupang.com/vp/products/21959462?itemId=85565373&vendorItemId=3145433102', 'http://www.coupang.com/vp/products

['http://www.coupang.com/vp/products/85604230?itemId=270702591&vendorItemId=3655706723', 'http://www.coupang.com/vp/products/92517388?itemId=287362790&vendorItemId=3705464996', 'http://www.coupang.com/vp/products/85604271?itemId=270702687&vendorItemId=3655706906', 'http://www.coupang.com/vp/products/85213422?itemId=269630675&vendorItemId=3653392987', 'http://www.coupang.com/vp/products/70221925?itemId=328078117&vendorItemId=3575015719', 'http://www.coupang.com/vp/products/85604255?itemId=270702647&vendorItemId=3655706833', 'http://www.coupang.com/vp/products/85604269?itemId=270702681&vendorItemId=3655706909', 'http://www.coupang.com/vp/products/70221927?itemId=234779974&vendorItemId=3575015720', 'http://www.coupang.com/vp/products/9699615?itemId=42854833&vendorItemId=3060297242', 'http://www.coupang.com/vp/products/19144393?itemId=76851176&vendorItemId=3129388632', 'http://www.coupang.com/vp/products/13060955?itemId=54807667&vendorItemId=3085806437', 'http://www.coupang.com/vp/products

In [40]:
links_diaper[1][0]

'http://www.coupang.com/vp/products/85604303?itemId=270702790&vendorItemId=3655707093'

In [41]:
links_diaper[2][0]

'http://www.coupang.com/vp/products/85604300?itemId=270702779&vendorItemId=3655707073'

In [42]:
links_diaper[3][0]

'http://www.coupang.com/vp/products/85604243?itemId=270702627&vendorItemId=3655706775'

In [44]:
diaperList = []

In [45]:
def getInfo_diaper(url, stage, num_retries=2):
    html = requests.get(url, headers=headers)
    
    if 500<=html.status_code<600 and num_retries>0:
        print(html.status_code, html.reason)
        return getInfo_diaper(seed, params, num_retries-1)
    
    dom = BeautifulSoup(html.text, 'lxml')
    if len(dom.select('div.oos-label')) == 0 and int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', '')) > 0:
        link = html.url
        img = 'http:' + dom.select('img.prod-image__detail')[0]['src']
        brand = dom.select('a.prod-brand-name')[0].get_text()
        title = dom.select('h2.prod-buy-header__title')[0].get_text()
        # product = 
        origin = 'unknown'
        for i in range(len(dom.select('ul.prod-description-attribute li'))):
            if '제조국' in dom.select('ul.prod-description-attribute li')[i].get_text():
                origin = dom.select('ul.prod-description-attribute li')[i].get_text().replace('제조국: ', '')
        stage = stage
        if '밴드형' in title:
            diaperType = '밴드형'
        elif '팬티형' in title:
            diaperType = '팬티형'
        elif '일자형' in title:
            diaperType = '일자형'
        else:
            return 0
        price = int(dom.select('span.total-price strong')[0].get_text().replace('\n', '').replace('원', '').replace(',', ''))
        replyCount = int(dom.select('a.moveAnchor span.count')[0].get_text().replace('개 상품평', '').replace(',', ''))
        
        driver.get(url)
        body = driver.find_element_by_tag_name('body')
        while True:
            try:
                driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
                break
            except NoSuchElementException:
                body.send_keys(Keys.PAGE_DOWN)
                driver.implicitly_wait(0.5)
        
        rating = driver.find_element_by_css_selector('.sdp-review__average__total-star__info-gray div')
        rating = rating.get_attribute('data-rating')
        replyResult = []
        buttonList = []
        buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
        
        def getReply(pagenum):
            try:
                element = WebDriverWait(driver, 5).until(lambda x: x.find_elements_by_css_selector('.js_reviewArticleListContainer article div.sdp-review__article__list__review__content.js_reviewArticleContent')) 
                replyResult.extend(reply.text.replace('\n', ' ') for reply in element)
                time.sleep(1)
            except TimeoutException:
                return None
            buttonList = []
            while True:
                try:
                    buttonList.extend(driver.find_elements_by_css_selector('.sdp-review__article__page__num.js_reviewArticlePageBtn'))
                    break
                except WebDriverException:
                    body.send_keys(Keys.PAGE_DOWN)
                    driver.implicitly_wait(0.5)
            if len(buttonList) > 0:
                buttonList[pagenum].click()
            else:
                return None
        
        if len(buttonList) > 0:
            end = len(buttonList)
            if end > 5:
                end = 5
            for i in range(1, end):
                getReply(i)
                time.sleep(0.5)
                
        #origin = driver.find_elements_by_css_selector('.prod-delivery-return-policy-table.essential-info-table tbody tr td')[1].text
    
        result = {
            'link': link, 
            'img': img, 
            'brand': brand,
            'origin': origin,
            'title': title,
            'stage': stage,
            'type': diaperType, 
            'price': price, 
            'rating': rating, 
            'optionTitle': None,
            'optionPrice': None,
            'replyCount': replyCount, 
            'replyResult': replyResult
        }
        diaperList.append(result)
        return 1
    
    else:
        return 0
    
    return None

In [46]:
for link in links_diaper[sizeList_diaper[sizeCategory[0]]]:
    getInfo_diaper(link, sizeList_diaper[sizeCategory[0]])
print(len(diaperList))

37


In [47]:
for link in links_diaper[sizeList_diaper[sizeCategory[1]]]:
    getInfo_diaper(link, sizeList_diaper[sizeCategory[1]])
print(len(diaperList))

94


In [48]:
for link in links_diaper[sizeList_diaper[sizeCategory[2]]]:
    getInfo_diaper(link, sizeList_diaper[sizeCategory[2]])
print(len(diaperList))

154


In [49]:
for link in links_diaper[sizeList_diaper[sizeCategory[3]]]:
    getInfo_diaper(link, sizeList_diaper[sizeCategory[3]])
print(len(diaperList))

212


In [57]:
for link in links_diaper[sizeList_diaper[sizeCategory[4]]]:
    getInfo_diaper(link, sizeList_diaper[sizeCategory[4]])
print(len(diaperList))

267


In [62]:
for link in links_diaper[sizeList_diaper[sizeCategory[5]]]:
    getInfo_diaper(link, sizeList_diaper[sizeCategory[5]])
print(len(diaperList))

321


In [63]:
len(diaperList)

321

___

데이터 형식 수정 완료. 다시 가져올것.(과자는 완료함)

In [64]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

In [65]:
coupang_db = client.get_database('coupang_db')

diaper_collection = coupang_db.get_collection('Diaper')

In [66]:
diaper_collection.insert_many([
    diaperList[i] for i in range(len(diaperList))
])